In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} pomegranate

/anaconda2/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,
Solving environment: \ 

In [ ]:
import pomegranate
#from pomegranate import BayesianNetwork

In [1]:
import numpy as np
import scipy
import networkx
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import notears
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [2]:
TRAIN = '../data/train/training_data.csv'
train_data = pd.read_csv(TRAIN, index_col=1)
dev_data = pd.read_csv('../data/dev/development_data.csv', index_col=1)

In [3]:
import re
from nltk.util import ngrams

def get_ngrams(n):
    n_grams = Counter()
    tweets = train_data[['tweet']]
    for row_index, row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        tokens = [token for token in s if token != ""]
        output = list(ngrams(tokens, n))
        n_grams = Counter(output)
    return n_grams


In [4]:
unis = get_ngrams(1)
top_unigrams = unis.most_common(100)
data = pd.concat([train_data, dev_data])

/Users/srizkall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def process_ngram_tweets(tweets, model):
    for word in [u[0] for u in model]:
        if len(word) > 1:
            tweets[word] = tweets['tweet'].str.contains(word).astype(int)
    word_counts = []
    swear_counts = []
    at_counts = []
    bad_words_set = set(open("bad-words.txt").read().split())

    for tweet in tweets['tweet']:
        tweet_words = tweet.split()
        word_counts.append(len(tweet))
        tot_bad = 0
        for word in tweet:            
            if word.lower() in bad_words_set:
                tot_bad+=1
        swear_counts.append(tot_bad)
        at_count = tweet.count('@')
        at_counts.append(at_count)

    tweets['Word Counts'] = word_counts
    tweets['Swear Counts'] = swear_counts
    tweets['@ Counts'] = at_counts
    return tweets[[col for col in tweets.columns if col!="tweet"]].values

In [6]:
tweets = data[['tweet']]
X_uni = process_ngram_tweets(tweets, "top_unigrams")

/Users/srizkall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/srizkall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/srizkall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [ ]:
X = tweets.values
model = BayesianNetwork.from_samples(X, algorithm='exact')